# Pandas API

Starting from version 3.2.0, Spark supports [Pandas API](https://spark.apache.org/docs/latest/api/python/getting_started/quickstart_ps.html). When using it, one can work with Spark DataFrames as if they were Pandas ones. Remember though, they never really are!

In [1]:
from pyspark.sql import SparkSession

# they say PyArrow improves Spark-pandas interoperability
spark = SparkSession.builder.config(
    "spark.sql.execution.arrow.pyspark.enabled", "true"
).getOrCreate()

22/06/03 19:21:36 WARN Utils: Your hostname, boris-work-laptop resolves to a loopback address: 127.0.1.1; using 192.168.1.2 instead (on interface wlp0s20f3)
22/06/03 19:21:36 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/06/03 19:21:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# we need to set this environment variable to make Spark happy
import os

os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"

In [3]:
from pyspark import pandas as pd

pd.set_option("compute.default_index_type", "distributed")
song = pd.read_json("song.json_lines")

In [4]:
song.head()

/home/boris/projects/AIUDSA18/venv/lib/python3.8/site-packages/pyspark/sql/pandas/conversion.py:87: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  Unsupported type in conversion to Arrow: ArrayType(StructType(List(StructField(end,DoubleType,true),StructField(label,StringType,true),StructField(start,DoubleType,true))),true)
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warnings.warn(msg)
22/06/03 19:22:02 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


,_id,albumTitle,album_genre,animux_paths,arousal,arousal_predicted,availableCountries,begin,bpm,chords_metadata,deezer_mapping,disambiguation,end,explicitLyrics,gain,has_emotion_tags,has_social_tags,id_album,id_album_deezer,id_artist_deezer,id_song_deezer,id_song_musicbrainz,isClassic,isrc,language,language_detect,lastfm_id,length,lengthAlbum,lyrics,multitrack_file,multitrack_path,name,position,preview,publicationDate,publicationDateAlbum,rank,rdf,summary,title,title_accent_fold,urlAllmusic,urlAmazon,urlDeezer,urlGoEar,urlHypeMachine,urlITunes,urlLastFm,urlMusicBrainz,urlPandora,urlSong,urlSpotify,urlWikipedia,urlYouTube,valence,valence_predicted
0,"(5714decb25ac0d8aee3ea246,)",Other Songs,,[],,-0.30319342,"[AD, AE, AF, AG, AI, AL, AM, AN, AO, AQ, AR, A...",,,None,"[[117385876, search-noalbum]]",,,False,,False,False,"{""$oid"":""5714debb25ac0d8aee3581d5""}",12185496,5580102,117385876,,False,FR6V81103654,,italian,,266,,Nu pu&#xF2; asc&#xEC; cu nat sol p&apos; pruva...,,,Daniele Bianco,12,http://e-cdn-preview-3.deezer.com/stream/3b4ef...,2011-01-25,,100000,,None,Te Fa' Male,Te Fa' Male,,,http://www.deezer.com/track/117385876,,,https://itunes.apple.com/us/album/id416809564?...,,,,http://lyrics.wikia.com/Daniele_Bianco:Te_Fa%2...,,,,,-0.14970228
1,"(5714decb25ac0d8aee3ea247,)",Other Songs,,[],,-0.22406061,"[AD, AE, AF, AG, AI, AL, AM, AN, AO, AQ, AR, A...",,,None,"[[117379786, search-noalbum]]",,,False,,False,False,"{""$oid"":""5714debb25ac0d8aee3581d5""}",12184916,5580102,117379786,,False,FR6V80643032,,italian,,213,,Sei davanti alla tv quasi ormai non dormi pi&#...,,,Daniele Bianco,13,http://e-cdn-preview-7.deezer.com/stream/71f0c...,2011-05-19,,100000,,None,Tu,Tu,,,http://www.deezer.com/track/117379786,,,https://itunes.apple.com/us/album/id438956469?...,,,,http://lyrics.wikia.com/Daniele_Bianco:Tu,,,,,-0.008508563
2,"(5714decb25ac0d8aee3ea248,)",Daniela Romo,Pop,[],,-0.18148870000000009,"[AD, AE, AF, AG, AI, AL, AM, AN, AO, AQ, AR, A...",,140,None,"[[3206617, search-exact], [3206617, sptfviaenki]]",,,False,-14.2,False,True,"{""$oid"":""5714debb25ac0d8aee3581d6""}",307106,243194,3206617,9b39c15a-cba2-40be-a0c3-72cfdba52875,False,ES5088302022,,spanish,TRSDWZZ128F425ED8E,231,34:23,Amarte a t&#xED; - es como adivinar - lo que d...,,,Daniela Romo,0,http://e-cdn-preview-b.deezer.com/stream/b7722...,2007-06-05,1983,421333,,None,Mentiras,Mentiras,,,http://www.deezer.com/track/3206617,,,https://itunes.apple.com/us/album/id714633783?...,,http://musicbrainz.org/recording/9b39c15a-cba2...,,http://lyrics.wikia.com/Daniela_Romo:Mentiras,https://play.spotify.com/track/2zyQxgik3Qdida6...,,,,0.0023454875
3,"(5714decb25ac0d8aee3ea249,)",Daniela Romo,Pop,[],,-0.16954462,"[AD, AE, AF, AG, AI, AL, AM, AN, AO, AQ, AR, A...",,130,None,"[[3430679, sptfviaenki], [3430725, search-exact]]",,,False,-12.8,False,False,"{""$oid"":""5714debb25ac0d8aee3581d6""}",325633,243194,3430679,24de11bc-20c3-4c0e-a726-bc0067974344,False,ES5088302748,,spanish,,218,34:23,T&#xFA; la impresi&#xF3;n que me d&#xE1;s - es...,,,Daniela Romo,1,http://e-cdn-preview-7.deezer.com/stream/788ea...,2007-07-17,1983,298451,,None,La Ocasión Para Amarnos,La Ocasion Para Amarnos,,,http://www.deezer.com/track/3430679,,,https://itunes.apple.com/us/album/id714642155?...,,http://musicbrainz.org/recording/24de11bc-20c3...,,http://lyrics.wikia.com/Daniela_Romo:La_Ocasi%...,https://play.spotify.com/track/64nsxDmHpvoTkMf...,,,,0.013402663
4,"(5714decb25ac0d8aee3ea24a,)",Daniela Romo,Pop,[],,-0.14722028,"[BZ, CR, GT, HN, MX, NI, PA, SV]",,169,None,"[[94173674, search-noalbum], [94173674, sptfvi...",,,False,-11.9,False,False,"{""$oid"":""5714debb25ac0d8aee3581d6""}",9573406,243194,94173674,cb333261-ca56-4199-9e5b-35d8d3fd9c72,False,ES5080800594,,spanish,,231,34:23,Desde hace tiempo no s&#xE9; - que me sucede -...,,,Daniela Romo,2,http://e-cdn-preview-6.deezer.com/stream/6fff6...,2015-02-10,1983,222878,,None,Te Amo,Te Amo,,,http://www.deezer.com/track/94173674,,,,,http://musicbrainz.org/recording/cb333

In [5]:
len(song)

2079510

# SQL API

In [6]:
# we can easily regiter a DataFrame as a table
song.spark.frame().createOrReplaceTempView("song")

In [7]:
# now we can write SQL queries as to any database
# notice that before we ask Spark to ``show``
# everything is a 'transformation', not an 'action'
# so it's like creating views
spark.sql("""
SELECT
    COUNT(*) AS cnt
FROM song
""").createOrReplaceTempView("count")

In [8]:
# here the action happens and we see the result set
spark.sql("SELECT * FROM count LIMIT 1").show()

+-------+
|    cnt|
+-------+
|2079510|
+-------+



# Spark DataFrames

In [9]:
# this API is rather different from both SQL and Pandas
# but provides most of the functionality
from pyspark.sql.functions import count_distinct
song_df = song.spark.frame()
song_df.select(
    count_distinct("rank").alias("number_of_ranks")
).show()

+---------------+
|number_of_ranks|
+---------------+
|          79650|
+---------------+



# Do It Yourself

[Spark Manual](https://spark.apache.org/docs/latest/api/python/reference) is your best friend!

* count numbers of albums, artists, countries, languages, genres
* find ten longest and sortest albumns (in terms of time and the number of songs)
* find top ten countries where the maximal number of songs is available
* build a table of total seconds of all songs appeared monthly through years
* find an album and an artist working in the largest number of different genres
* find ten most frequent words in ten most popular languages

When done with your favourite API, try doing the same with another one!